In [1]:
from google.colab import drive

drive.mount("/content/gdrive")
!pwd  # show current path

Mounted at /content/gdrive
/content


In [2]:
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos
Base			Dimensiones  Predicciones  Test		  tVenta.csv
CaracteristicasCliente	Fact	     tCliente.csv  tProducto.csv


In [3]:
import pandas as pd

# Generacion Tablas Base

In [4]:
tCliente = pd.read_csv("tCliente.csv")
tVenta = pd.read_csv("tVenta.csv")
tProducto = pd.read_csv("tProducto.csv")

In [5]:
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Dimensiones"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Dimensiones
dimContenedor.csv  dimNivelSociEconomico.csv  dimProductoTipo.csv  dimSubCanal.csv  dimZone.csv
dimMarca.csv	   dimProductoCategoria.csv   dimSabor.csv	   dimTamano.csv


In [6]:
dimProductoCategoria = pd.read_csv("dimProductoCategoria.csv")
dimProductoTipo = pd.read_csv("dimProductoTipo.csv")

In [7]:
#Despues de agarrar datos Base se mueve al directorio a datos
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Fact"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Fact
factClienteCompraVariacion.csv		     factClienteVentaCategoria.csv
factClienteCompraVariacionTipo.csv	     factClienteVentaTipo.csv
factClienteProductoVentaPromedioMensual.csv  tProductoClienteExitoso.csv


In [8]:
tProductoClienteExitoso = pd.read_csv("tProductoClienteExitoso.csv")

In [9]:
tCliente = tCliente.astype({'CustomerId': 'int'})
tVenta = tVenta.astype({'CustomerId': 'int', 'Material': "int",  "uni_box": "float"})
tVenta["calmonth"] = pd.to_datetime(tVenta["calmonth"])
tProducto = tProducto.astype({'Material': "int"})

tVenta = tVenta.rename(columns={"material": "Material"})


# Fact Clientes Compra Categorias
Indices de compra de cada 3 meses de cada cliente de los diferentes Categorias que se encuentran en la columna product_category

In [10]:


# Primer merge: ventas con productos basado en la columna 'Material'
tVentaProducto = pd.merge(tVenta, tProducto, on='Material', how='inner')

# Segundo merge: el resultado con customers usando la columna 'CustomerId'
tVentaProductoCliente = pd.merge(tVentaProducto, tCliente, on='CustomerId', how='inner').merge(dimProductoCategoria, on='iIdProductoCategoria', how='inner')

# Asegurarse de que 'calmonth' sea tipo datetime para facilitar el filtrado
tVentaProductoCliente['calmonth'] = pd.to_datetime(tVentaProductoCliente['calmonth'], format='%Y%m')

# Ordenar el DataFrame por fecha para facilitar la agrupación
tVentaProductoCliente = tVentaProductoCliente.sort_values(by='calmonth')

# Definir el primer mes y el último mes en los datos
dFirstMonth = tVentaProductoCliente['calmonth'].min()
dLastMonth = tVentaProductoCliente['calmonth'].max()

# Crear una lista para almacenar los resultados de cada trimestre
oResultadosTrimestrales = []

# Iterar sobre los trimestres y agrupar las ventas por CustomerId y ProductCategory
while dFirstMonth < dLastMonth:
    # Definir el final del trimestre
    dTrimestreEnd = dFirstMonth + pd.DateOffset(months=6)

    # Filtrar las ventas de este trimestre
    tVentaTrimestre = tVentaProductoCliente[(tVentaProductoCliente['calmonth'] >= dFirstMonth) &
                                                (tVentaProductoCliente['calmonth'] < dTrimestreEnd)]

    # Agrupar las ventas por CustomerId y ProductCategory, sumando las unidades
    tVentaProductoTrimestre = tVentaTrimestre.groupby(['CustomerId', 'sProductoCategoria'])['uni_box'].sum().reset_index()

    # Agregar una columna indicando el periodo del trimestre
    tVentaProductoTrimestre['Periodo'] = f'{dFirstMonth.strftime("%Y-%m")} a {dTrimestreEnd.strftime("%Y-%m")}'

    # Agregar los resultados a la lista
    oResultadosTrimestrales.append(tVentaProductoTrimestre)

    # Avanzar al siguiente trimestre
    dFirstMonth = dTrimestreEnd

# Concatenar todos los trimestres en un solo DataFrame
tVentaTrimestral = pd.concat(oResultadosTrimestrales, ignore_index=True)

# Pivotar la tabla para obtener el formato deseado
tVentaTrimestralPivot = tVentaTrimestral.pivot_table(
    index='CustomerId',
    columns='sProductoCategoria',
    values='uni_box',  # Usar 'uni_box' para los valores
    aggfunc='mean',  # Cambiar a 'mean' para obtener el promedio
    fill_value=0  # Reemplazar NaN con 0
).reset_index()

#tVentaTrimestral.drop(columns=['REFRESCOS'], inplace=True)
# Renombrar las columnas si es necesario (opcional)
tVentaTrimestralPivot.columns.name = None  # Quitar el nombre de la columna
tVentaTrimestralPivot = tVentaTrimestralPivot.rename_axis(None, axis=1)  # Quitar el nombre del índice

# Formatear las columnas a punto decimal
tVentaTrimestralPivot = tVentaTrimestralPivot.applymap(lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x)

# Mostrar los resultados agrupados
tVentaTrimestralPivot.describe()

# Guardar los resultados en un archivo CSV
tVentaTrimestralPivot.to_csv('factClienteVentaCategoria.csv', index=False)

<ipython-input-10-426637358fc7>:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tVentaTrimestralPivot = tVentaTrimestralPivot.applymap(lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x)


In [11]:
tVentaTrimestral.groupby(['CustomerId', 'sProductoCategoria'])['uni_box'].mean().reset_index()

CustomerId      sProductoCategoria     uni_box
0       499920078                    AGUA  115.979617
1       499920078  AGUA MINERAL CON SABOR    1.479333
2       499920078          AGUA SALUDABLE   27.544317
3       499920078    BEBIDAS ALTERNATIVAS    1.053600
4       499920078     BEBIDAS CON ALCOHOL    1.250333
...           ...                     ...         ...
13759   510767085                 LACTEOS    0.140900
13760   510767085               REFRESCOS   27.333600
13761   510767623                    AGUA    1.056700
13762   510767623                 LACTEOS    1.056700
13763   510767623               REFRESCOS  128.481700

[13764 rows x 3 columns]

In [12]:
tVentaTrimestralPivot

CustomerId    AGUA AGUA MINERAL CON SABOR AGUA SALUDABLE  \
0     499920078.00  115.98                   1.48          27.54   
1     499920499.00  112.14                   3.14          57.22   
2     499921473.00  191.47                   5.20          25.92   
3     499921557.00  303.14                   7.03         116.78   
4     499921908.00  338.90                   2.29          77.90   
...            ...     ...                    ...            ...   
1925  510764476.00    4.23                   0.00           0.00   
1926  510765506.00   18.74                   0.00           1.69   
1927  510766660.00    6.48                   0.00           3.28   
1928  510767085.00    0.00                   0.00           1.90   
1929  510767623.00    1.06                   0.00           0.00   

     BEBIDAS ALTERNATIVAS BEBIDAS CON ALCOHOL CAFES JUGOS Y NECTARES LACTEOS  \
0                    1.05                1.25  0.00            31.59    8.85   
1                    6.11                0.38  1.23            31.69    2.12   
2                    5.72                3.25  0.38            66.64    9.11   
3                   10.93                0.00  1.85           269.54   26.81   
4                   22.20                0.75  0.15           107.96   12.58   
...                   ...                 ...   ...              ...     ...   
1925                 0.33                0.00  0.00             2.47    0.00   
1926                 1.40                0.00  1.23            25.02    3.17   
1927                 1.04                0.00  0.30            17.02    0.77   
1928                 1.39                0.00  0.00             6.94    0.14   
1929                 0.00                0.00  0.00             0.00    1.06   

     REFRESCOS  
0       771.13  
1      1522.55  
2      3053.24  
3      2581.79  
4      1811.47  
...        ...  
1925     63.95  
1926    220.00  
1927    220.01  
1928     27.33  
1929    128.48  

[1930 rows x 10 columns]

In [13]:
tVenta[tVenta["CustomerId"] == 499921473]

CustomerId  Material   calmonth  uni_box dPrimeraCompraCliente
2563   499921473       372 2019-09-01  28.1792            2019-09-01
2564   499921473      5001 2019-09-01   1.9866            2019-09-01
2565   499921473       117 2019-09-01   2.1134            2019-09-01
2566   499921473      9460 2019-09-01   1.0567            2019-09-01
2567   499921473       526 2019-09-01   1.5851            2019-09-01
...          ...       ...        ...      ...                   ...
4407   499921473       766 2022-06-01   0.6340            2019-09-01
4408   499921473     14525 2022-06-01   0.3751            2019-09-01
4409   499921473       405 2022-06-01   5.0722            2019-09-01
4410   499921473      2287 2022-06-01  41.2113            2019-09-01
4411   499921473       520 2022-06-01   6.3402            2019-09-01

[1849 rows x 5 columns]

# Fact Cliente Compra Tipo

In [14]:


# Primer merge: ventas con productos basado en la columna 'Material'
tVentaProducto = pd.merge(tVenta, tProducto, on='Material', how='inner')

# Segundo merge: el resultado con customers usando la columna 'CustomerId'
tVentaProductoCliente = pd.merge(tVentaProducto, tCliente, on='CustomerId', how='inner').merge(dimProductoTipo, on='iIdProductoTipo', how='inner')

# Asegurarse de que 'calmonth' sea tipo datetime para facilitar el filtrado
tVentaProductoCliente['calmonth'] = pd.to_datetime(tVentaProductoCliente['calmonth'], format='%Y%m')

# Ordenar el DataFrame por fecha para facilitar la agrupación
tVentaProductoCliente = tVentaProductoCliente.sort_values(by='calmonth')

# Definir el primer mes y el último mes en los datos
dFirstMonth = tVentaProductoCliente['calmonth'].min()
dLastMonth = tVentaProductoCliente['calmonth'].max()

# Crear una lista para almacenar los resultados de cada trimestre
oResultadosTrimestrales = []

# Iterar sobre los trimestres y agrupar las ventas por CustomerId y ProductCategory
while dFirstMonth < dLastMonth:
    # Definir el final del trimestre
    dTrimestreEnd = dFirstMonth + pd.DateOffset(months=6)

    # Filtrar las ventas de este trimestre
    tVentaTrimestre = tVentaProductoCliente[(tVentaProductoCliente['calmonth'] >= dFirstMonth) &
                                                (tVentaProductoCliente['calmonth'] < dTrimestreEnd)]

    # Agrupar las ventas por CustomerId y ProductCategory, sumando las unidades
    tVentaProductoTrimestre = tVentaTrimestre.groupby(['CustomerId', 'sProductoTipo'])['uni_box'].sum().reset_index()

    # Agregar una columna indicando el periodo del trimestre
    tVentaProductoTrimestre['Periodo'] = f'{dFirstMonth.strftime("%Y-%m")} a {dTrimestreEnd.strftime("%Y-%m")}'

    # Agregar los resultados a la lista
    oResultadosTrimestrales.append(tVentaProductoTrimestre)

    # Avanzar al siguiente trimestre
    dFirstMonth = dTrimestreEnd

# Concatenar todos los trimestres en un solo DataFrame
tVentaTrimestral = pd.concat(oResultadosTrimestrales, ignore_index=True)

# Pivotar la tabla para obtener el formato deseado
tVentaTrimestralPivot = tVentaTrimestral.pivot_table(
    index='CustomerId',
    columns='sProductoTipo',
    values='uni_box',  # Usar 'uni_box' para los valores
    aggfunc='mean',  # Cambiar a 'mean' para obtener el promedio
    fill_value=0  # Reemplazar NaN con 0
).reset_index()

#tVentaTrimestral.drop(columns=['REFRESCOS'], inplace=True)
# Renombrar las columnas si es necesario (opcional)
tVentaTrimestralPivot.columns.name = None  # Quitar el nombre de la columna
tVentaTrimestralPivot = tVentaTrimestralPivot.rename_axis(None, axis=1)  # Quitar el nombre del índice

# Formatear las columnas a punto decimal
tVentaTrimestralPivot = tVentaTrimestralPivot.applymap(lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x)

# Mostrar los resultados agrupados
tVentaTrimestralPivot.describe()

# Guardar los resultados en un archivo CSV
tVentaTrimestralPivot.to_csv('factClienteVentaTipo.csv', index=False)

<ipython-input-14-fbed7813262b>:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tVentaTrimestralPivot = tVentaTrimestralPivot.applymap(lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x)


In [15]:
tVentaTrimestralPivot.columns

Index(['CustomerId', 'AGUA FUNCIONAL', 'AGUA MINERAL', 'AGUA PURIFICADA',
       'AGUA SABORIZADA', 'BEBIDA ALCOHOLICA', 'BEBIDAS DE SOYA',
       'BEBIDAS ENERGETICAS', 'BEBIDAS INFANTILES', 'BEBIDAS REFRESCANTES',
       'CAFE GRANO', 'CAFE MOLIDO', 'COLAS LIGHT', 'COLAS REGULAR',
       'ISOTONICOS', 'JUGOS Y NECTARES', 'LECHE UHT ESPECIALIZADA',
       'LECHE UHT REGULAR', 'LECHE UHT SABORIZADA', 'MIXTOS', 'NARANJADAS',
       'SABORES LIGHT', 'SABORES REGULAR', 'TE'],
      dtype='object')

# FactClientesCompraCategoriaVariacion

In [16]:
tVentaProducto = pd.merge(tVenta, tProducto, on='Material', how='inner')
#tVentaProducto = tVenta.merge(tProducto, on='Material', how='inner')


In [17]:
tVentaProducto.columns

Index(['CustomerId', 'Material', 'calmonth', 'uni_box',
       'dPrimeraCompraCliente', 'Material_desc', 'Productos_Por_Empaque',
       'BrandPresRet', 'ProdKey', 'Presentation', 'MLSize', 'Pack', 'Ncb',
       'SegAg', 'SegDet', 'GlobalCategory', 'GlobalSubcategory',
       'BrandGrouper', 'GlobalFlavor', 'dPrimeraVenta', 'iIdProductoTipo',
       'iIdContenedor', 'iIdMarca', 'iIdProductoCategoria', 'iIdTamano',
       'iIdSabor', 'bReturnability', 'bViejo', 'bExitoso',
       'iIdClusterProducto'],
      dtype='object')

In [18]:


tClienteVenta = tVentaProducto.groupby('CustomerId', as_index=False)['uni_box'].sum()


tClienteProductoUnico = tVentaProducto.groupby(['CustomerId', "iIdProductoCategoria"])['Material_desc'].nunique().reset_index()
tClienteProductoUnico = tClienteProductoUnico.merge(dimProductoCategoria, on='iIdProductoCategoria', how='inner' )


tClienteProductoUnico = tClienteProductoUnico.pivot_table(index='CustomerId', columns='sProductoCategoria', values='Material_desc', aggfunc='sum').reset_index()
tClienteProductoUnico = tClienteProductoUnico.fillna(0)
tClienteProductoUnico = tClienteProductoUnico.merge(tClienteVenta, on='CustomerId', how='inner' )


tClienteProductoUnico = tClienteProductoUnico[['CustomerId', 'uni_box', 'AGUA', 'AGUA MINERAL CON SABOR', 'AGUA SALUDABLE',
                                              'BEBIDAS ALTERNATIVAS', 'BEBIDAS CON ALCOHOL', 'CAFES',
                                              'JUGOS Y NECTARES', 'LACTEOS', 'REFRESCOS'] ]
tClienteProductoUnico.to_csv('factClienteCompraVariacion.csv', index=False)
tClienteProductoUnico.describe()


CustomerId        uni_box         AGUA  AGUA MINERAL CON SABOR  \
count  1.944000e+03    1944.000000  1944.000000             1944.000000   
mean   5.042451e+08    8136.491595    10.577675                1.187243   
std    5.003736e+06   20340.138060     5.125309                1.054326   
min    4.999201e+08       0.176100     0.000000                0.000000   
25%    5.001446e+08    1973.519639     7.000000                0.000000   
50%    5.003950e+08    4647.261058    10.000000                1.000000   
75%    5.103485e+08    9813.760381    14.000000                2.000000   
max    5.107817e+08  730532.970127    29.000000                5.000000   

       AGUA SALUDABLE  BEBIDAS ALTERNATIVAS  BEBIDAS CON ALCOHOL        CAFES  \
count     1944.000000           1944.000000          1944.000000  1944.000000   
mean         6.489198              9.145062             0.484053     0.651235   
std          3.946358              5.503711             1.365237     0.712919   
min          0.000000              0.000000             0.000000     0.000000   
25%          4.000000              5.000000             0.000000     0.000000   
50%          6.000000              9.000000             0.000000     1.000000   
75%          9.000000             13.000000             0.000000     1.000000   
max         20.000000             29.000000             8.000000     4.000000   

       JUGOS Y NECTARES      LACTEOS    REFRESCOS  
count       1944.000000  1944.000000  1944.000000  
mean          21.238169     6.238683    50.583333  
std           12.103489     4.157430    25.043950  
min            0.000000     0.000000     0.000000  
25%           12.000000     3.000000    31.000000  
50%           20.000000     6.000000    49.000000  
75%           29.000000     9.000000    68.250000  
max           59.000000    19.000000   135.000000

# Fact Cliente Compra Tipo Variacion

In [19]:


tClienteVenta = tVentaProducto.groupby('CustomerId', as_index=False)['uni_box'].sum()


tClienteProductoUnico = tVentaProducto.groupby(['CustomerId', "iIdProductoTipo"])['Material_desc'].nunique().reset_index()
tClienteProductoUnico = tClienteProductoUnico.merge(dimProductoTipo, on='iIdProductoTipo', how='inner' )


tClienteProductoUnico = tClienteProductoUnico.pivot_table(index='CustomerId', columns='sProductoTipo', values='Material_desc', aggfunc='sum').reset_index()
tClienteProductoUnico = tClienteProductoUnico.fillna(0)
tClienteProductoUnico = tClienteProductoUnico.merge(tClienteVenta, on='CustomerId', how='inner' )



tClienteProductoUnico = tClienteProductoUnico[['CustomerId', 'uni_box', 'AGUA FUNCIONAL', 'AGUA MINERAL', 'AGUA PURIFICADA',
                                                'AGUA SABORIZADA', 'BEBIDA ALCOHOLICA', 'BEBIDAS DE SOYA',
                                                'BEBIDAS ENERGETICAS', 'BEBIDAS INFANTILES', 'BEBIDAS REFRESCANTES',
                                                'CAFE GRANO', 'CAFE MOLIDO', 'COLAS LIGHT', 'COLAS REGULAR',
                                                'ISOTONICOS', 'JUGOS Y NECTARES', 'LECHE UHT ESPECIALIZADA',
                                                'LECHE UHT REGULAR', 'LECHE UHT SABORIZADA', 'MIXTOS', 'NARANJADAS',
                                                'SABORES LIGHT', 'SABORES REGULAR', 'TE'] ]
tClienteProductoUnico.to_csv('factClienteCompraVariacionTipo.csv', index=False)
tClienteProductoUnico.describe()

CustomerId        uni_box  AGUA FUNCIONAL  AGUA MINERAL  \
count  1.944000e+03    1944.000000     1944.000000   1944.000000   
mean   5.042451e+08    8136.491595        0.211420      4.624486   
std    5.003736e+06   20340.138060        0.678357      2.733475   
min    4.999201e+08       0.176100        0.000000      0.000000   
25%    5.001446e+08    1973.519639        0.000000      3.000000   
50%    5.003950e+08    4647.261058        0.000000      4.000000   
75%    5.103485e+08    9813.760381        0.000000      6.000000   
max    5.107817e+08  730532.970127        4.000000     18.000000   

       AGUA PURIFICADA  AGUA SABORIZADA  BEBIDA ALCOHOLICA  BEBIDAS DE SOYA  \
count      1944.000000      1944.000000        1944.000000      1944.000000   
mean          4.788580         2.351852           0.484053         3.020062   
std           2.058849         2.355154           1.365237         2.666795   
min           0.000000         0.000000           0.000000         0.000000   
25%           4.000000         0.000000           0.000000         1.000000   
50%           5.000000         2.000000           0.000000         3.000000   
75%           6.000000         4.000000           0.000000         5.000000   
max          12.000000        12.000000           8.000000        12.000000   

       BEBIDAS ENERGETICAS  BEBIDAS INFANTILES  ...   ISOTONICOS  \
count          1944.000000         1944.000000  ...  1944.000000   
mean              4.181584            1.943416  ...     6.277778   
std               2.555901            1.523833  ...     3.680358   
min               0.000000            0.000000  ...     0.000000   
25%               2.000000            0.000000  ...     4.000000   
50%               4.000000            2.000000  ...     6.000000   
75%               6.000000            3.000000  ...     9.000000   
max              16.000000            6.000000  ...    17.000000   

       JUGOS Y NECTARES  LECHE UHT ESPECIALIZADA  LECHE UHT REGULAR  \
count       1944.000000              1944.000000        1944.000000   
mean          12.270576                 0.983539           1.617798   
std            7.599323                 1.197328           1.230724   
min            0.000000                 0.000000           0.000000   
25%            7.000000                 0.000000           1.000000   
50%           11.000000                 1.000000           1.500000   
75%           17.000000                 2.000000           2.000000   
max           41.000000                 6.000000           7.000000   

       LECHE UHT SABORIZADA       MIXTOS   NARANJADAS  SABORES LIGHT  \
count           1944.000000  1944.000000  1944.000000    1944.000000   
mean               3.485597     0.151749     4.200103       0.207819   
std                2.247844     0.358870     2.374721       0.569440   
min                0.000000     0.000000     0.000000       0.000000   
25%                2.000000     0.000000     2.000000       0.000000   
50%                4.000000     0.000000     4.000000       0.000000   
75%                5.000000     0.000000     6.000000       0.000000   
max               10.000000     1.000000    12.000000       4.000000   

       SABORES REGULAR           TE  
count      1944.000000  1944.000000  
mean         26.556584     2.811214  
std          16.659772     2.310855  
min           0.000000     0.000000  
25%          13.000000     1.000000  
50%          25.000000     3.000000  
75%          38.000000     5.000000  
max          90.000000     9.000000  

[8 rows x 25 columns]

In [20]:
tClienteProductoUnico.columns

Index(['CustomerId', 'uni_box', 'AGUA FUNCIONAL', 'AGUA MINERAL',
       'AGUA PURIFICADA', 'AGUA SABORIZADA', 'BEBIDA ALCOHOLICA',
       'BEBIDAS DE SOYA', 'BEBIDAS ENERGETICAS', 'BEBIDAS INFANTILES',
       'BEBIDAS REFRESCANTES', 'CAFE GRANO', 'CAFE MOLIDO', 'COLAS LIGHT',
       'COLAS REGULAR', 'ISOTONICOS', 'JUGOS Y NECTARES',
       'LECHE UHT ESPECIALIZADA', 'LECHE UHT REGULAR', 'LECHE UHT SABORIZADA',
       'MIXTOS', 'NARANJADAS', 'SABORES LIGHT', 'SABORES REGULAR', 'TE'],
      dtype='object')

# Fact Cliente Producto Compra Mensual

In [21]:
tVentaMensual = tVenta.groupby(["CustomerId", "Material", "calmonth"])["uni_box"].sum().reset_index()

tMeses = pd.MultiIndex.from_product(
    [tVenta["CustomerId"].unique(), tVenta["Material"].unique(), tVenta["calmonth"].unique()],
    names=["CustomerId", "Material", "calmonth"]
).to_frame(index=False)

tVentaMensualCompleta = tMeses.merge(
    tVentaMensual,
    on=["CustomerId", "Material", "calmonth"],
    how="left"
).fillna(0)


# Calcular el promedio mensual de ventas para cada combinación de CustomerId y Material
factClienteProductoVentaPromedio = tVentaMensualCompleta.groupby(["CustomerId", "Material"])["uni_box"].mean().reset_index()

# Renombrar la columna para indicar que es el promedio mensual de ventas
factClienteProductoVentaPromedioMensual = factClienteProductoVentaPromedio.rename(columns={"uni_box": "promedio_venta_mensual"})


In [22]:
factClienteProductoVentaPromedioMensual.to_csv('factClienteProductoVentaPromedioMensual.csv', index=False)
factClienteProductoVentaPromedioMensual.describe()

CustomerId      Material  promedio_venta_mensual
count  1.389960e+06  1.389960e+06            1.389960e+06
mean   5.042451e+08  6.536140e+03            3.346973e-01
std    5.002450e+06  5.385188e+03            9.114489e+00
min    4.999201e+08  1.000000e+00           -2.641750e-01
25%    5.001446e+08  1.268000e+03            0.000000e+00
50%    5.003950e+08  4.884000e+03            0.000000e+00
75%    5.103485e+08  9.882000e+03            0.000000e+00
max    5.107817e+08  1.736900e+04            4.268592e+03

# Fact Cliente Compra Producto Exitoso Tipo

In [46]:
tClienteVenta = tVentaProducto.groupby(['CustomerId', "Material"], as_index=False)['uni_box'].sum()

tClienteProductoExitosoVenta = tProductoClienteExitoso.merge(tClienteVenta, on=['CustomerId', "Material"], how='left')

tClienteProductoExitosoVenta = tClienteProductoExitosoVenta.merge(tProducto, on='Material', how='inner')

factClienteCompraVariacionTipoExitoso = tClienteProductoExitosoVenta.groupby(['CustomerId', "iIdProductoTipo"])['Material_desc'].nunique().reset_index().merge(dimProductoTipo, on='iIdProductoTipo', how='inner' ).pivot_table(index='CustomerId', columns='sProductoTipo', values='Material_desc', aggfunc='sum').reset_index().fillna(0)
factClienteCompraVariacionTipoExitoso.to_csv('factClienteCompraVariacionTipoExitoso.csv', index=False)
factClienteCompraVariacionTipoExitoso.describe()

sProductoTipo    CustomerId  AGUA FUNCIONAL  AGUA MINERAL  AGUA PURIFICADA  \
count          1.545000e+03     1545.000000    1545.00000      1545.000000   
mean           5.027664e+08        0.009709       0.68932         0.964401   
std            4.390421e+06        0.162666       0.84209         1.207447   
min            4.999201e+08        0.000000       0.00000         0.000000   
25%            5.001108e+08        0.000000       0.00000         0.000000   
50%            5.002918e+08        0.000000       1.00000         0.000000   
75%            5.099777e+08        0.000000       1.00000         2.000000   
max            5.106976e+08        4.000000       5.00000         6.000000   

sProductoTipo  AGUA SABORIZADA  BEBIDA ALCOHOLICA  BEBIDAS DE SOYA  \
count              1545.000000        1545.000000      1545.000000   
mean                  0.083495           0.003883         0.097087   
std                   0.389490           0.062217         0.486500   
min                   0.000000           0.000000         0.000000   
25%                   0.000000           0.000000         0.000000   
50%                   0.000000           0.000000         0.000000   
75%                   0.000000           0.000000         0.000000   
max                   6.000000           1.000000         5.000000   

sProductoTipo  BEBIDAS ENERGETICAS  BEBIDAS INFANTILES  BEBIDAS REFRESCANTES  \
count                  1545.000000         1545.000000           1545.000000   
mean                      0.321036            0.400000              0.425243   
std                       0.694523            0.974015              0.767548   
min                       0.000000            0.000000              0.000000   
25%                       0.000000            0.000000              0.000000   
50%                       0.000000            0.000000              0.000000   
75%                       0.000000            0.000000              1.000000   
max                       4.000000            4.000000              4.000000   

sProductoTipo  ...   ISOTONICOS  JUGOS Y NECTARES  LECHE UHT ESPECIALIZADA  \
count          ...  1545.000000       1545.000000              1545.000000   
mean           ...     1.280259          2.341100                 0.055016   
std            ...     1.923681          2.981293                 0.262417   
min            ...     0.000000          0.000000                 0.000000   
25%            ...     0.000000          0.000000                 0.000000   
50%            ...     0.000000          1.000000                 0.000000   
75%            ...     2.000000          4.000000                 0.000000   
max            ...    13.000000         20.000000                 2.000000   

sProductoTipo  LECHE UHT REGULAR  LECHE UHT SABORIZADA       MIXTOS  \
count                1545.000000           1545.000000  1545.000000   
mean                    0.214887              0.415534     0.005825   
std                     0.420228              0.966816     0.076125   
min                     0.000000              0.000000     0.000000   
25%                     0.000000              0.000000     0.000000   
50%                     0.000000              0.000000     0.000000   
75%                     0.000000              0.000000     0.000000   
max                     2.000000              8.000000     1.000000   

sProductoTipo   NARANJADAS  SABORES LIGHT  SABORES REGULAR           TE  
count          1545.000000    1545.000000      1545.000000  1545.000000  
mean              1.240777       0.038835         3.978641     0.427832  
std               1.492301       0.193264         5.360833     0.885331  
min               0.000000       0.000000         0.000000     0.000000  
25%               0.000000       0.000000         0.000000     0.000000  
50%               1.000000       0.000000         2.000000     0.000000  
75%               2.000000       0.000000         6.000000     1.000000  
max       